In [ ]:
import numpy as np
import pandas as pd
# import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import minmax_scale

In [ ]:
# Load the 5-year time series data of 8 variables into a pandas DataFrame
df = pd.read_csv('vix_data.csv')
df['Date'] = pd.to_datetime(df['Date'], format = '%d/%m/%Y')
df=df.fillna(method='bfill')
df = df.drop(['Date','(R1) Open', '(R1) High', '(R1) Low'], axis=1)

In [ ]:
# Normalize the data using MinMaxScaler
#scaler = MinMaxScaler(feature_range=(0, 1))
#data = scaler.fit_transform(df.values)

cols = [x for x in df.columns if x != 'NIFTY 30d volatility']
df[cols] = minmax_scale(df[cols])
df.drop(index=df.index[0], axis=0, inplace=True)
data = df.values

In [ ]:
df.iloc[0]

In [ ]:
data.shape

Total  = 1238 * 8

Input length  = 400 * 8

Output length  = 7 * 1 : VIX for next week

We are taking data of last 400 days to predict next 7 days VIX

For training we will make overlaping training samples shifted by 'n_shift' values

In [ ]:
# Create input-output pairs from the data
timesteps = 400 # number of timesteps in the input sequence (1 year)
features = 8 # number of features in each timestep
output_timesteps = 7 # number of timesteps in the output sequence (7 days)
n_shift = 50
X, y = [], []
for i in range(len(data) - timesteps - output_timesteps + 1):
  if i%n_shift==0:
      X.append(data[i:i+timesteps, :features])
      y.append(data[i+timesteps:i+timesteps+output_timesteps, 2])


In [ ]:
# Convert the input-output pairs to numpy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# Split the data into training and testing sets
train_size = 10
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

In [ ]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train.shape[1]
n_timesteps, n_features, n_outputs

In [ ]:
# Define the model
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)
model = Sequential()
model.add(LSTM(128, input_shape=(n_timesteps, n_features)))
# model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Dense(n_outputs))
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
# Train the model on the training data
model.fit(X_train, y_train, epochs=100, batch_size=1, verbose=2, callbacks=[callback])

In [ ]:
# Evaluate the model on the test data
test_loss = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}')

# Use the trained model to make predictions
predictions = model.predict(X_test)

In [ ]:
predictions.shape

In [ ]:
predictions

In [ ]:
y_test

In [ ]:
(np.abs(predictions-y_test)).mean()